# [TECH CHALLENGE] Fine-Tuning de Modelos de LLM com o Dataset "The AmazonTitles-1.3MM"

> Add blockquote



> Turma **1IADT**
>
> Fase: **3 - Open AI**
>
> Grupo: **37**
> Participantes:
> - IGOR DE SOUZA SILVA
> - JOÃO GABRIEL COLARES DE CAMARGOS
> - MAXIMILIANO DE OLIVEIRA FURTADO

## 1. Visão Geral
Este documento descreve o processo de fine-tuning de um modelo LLM (Large Language Model), utilizando o dataset "The AmazonTitles-1.3MM", com o objetivo de configurar o modelo para responder a perguntas dos usuários. O modelo deve gerar respostas com base nas perguntas recebidas, utilizando o conhecimento adquirido durante o fine-tuning e incluindo informações relevantes das fontes fornecidas no dataset.

## 2. Preparação do Ambiente

### 2.1. Requisitos
Para executar o processo de fine-tuning e configurar o modelo para responder a perguntas, certifique-se de que o ambiente possui as seguintes ferramentas e bibliotecas:

- **Python 3.x**
- **Bibliotecas**:
  - `transformers` (para trabalhar com o **TextStreamer** e outras funcionalidades do Hugging Face)
  - `datasets` (para carregar o dataset)
  - `torch` (PyTorch para operações de deep learning)
  - `pandas` (para manipulação de dados)
  - `unsloth` (para utilizar o **FastLanguageModel** e verificar suporte a bfloat16)
  - `trl` (para utilizar o **SFTTrainer** e realizar o fine-tuning)
- **GPU**: Idealmente, uma GPU com suporte a CUDA para acelerar o treinamento.
- **Google Drive**: Acesso ao Google Drive, onde deverá encontra-se o dataset, além de servir de armazenamento para o modelo treinado.

#### 2.1.1 Por que `unsloth`?
Unsloth é uma plataforma que facilita o ajuste fino e o pretreinamento contínuo de grandes modelos de linguagem (LLMs), como Llama-3 e Mistral, com alta eficiência. Ela permite treinar modelos 2 vezes mais rápido e reduzir o consumo de VRAM em até 50%, usando técnicas como quantização em 4 bits. Ideal para adaptar modelos a novos domínios de conhecimento, o Unsloth é eficaz em ambientes de hardware limitado ou com grandes volumes de dados, garantindo economia de recursos sem sacrificar a performance​ *(como é o caso deste projeto)*.

### 2.2. Instalação das Dependências e Acesso ao Google Drive

Execute os seguintes comandos para instalar as dependências e permitir que o Collab tenha acesso ao Drive:

In [ ]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install transformers datasets

### 2.3 Importações Necessárias

Para este desafio, é necessário utilizar os seguintes imports:


In [ ]:
import torch
import pandas as pd
from datasets import load_dataset
from transformers import TextStreamer, TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## 3. Processo de Fine-Tuning

### 3.1. Escolha do Modelo
O código abaixo carrega o modelo e o tokenizer a partir de um modelo pré-treinado `Llama 3.1` na sua versão de 4 bits.

**Parâmetros**:
- `model_name`: Nome do modelo pré-treinado a ser carregado.
- `max_seq_length`: Comprimento máximo da sequência para o modelo.
- `dtype`: Tipo de dado (por padrão, None).
- `load_in_4bit`: Carregar o modelo em formato 4-bit para otimização de memória.



In [ ]:
max_seq_length = 128

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

### 3.2 Teste com o Modelo Pré-treinado
Nesta etapa, realizamos testes com o modelo base (Llama) antes de iniciar o processo de fine-tuning com o dataset específico. Como previsto, os resultados obtidos não apresentaram respostas satisfatórias ou contextualizadas, refletindo o comportamento padrão de um modelo ainda não especializado no domínio dos dados.



In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Answer the user's question", # instruction
        "What product Adult Ballet Tutu Yellow is?", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the user's question

### Input:
What product Adult Ballet Tutu Yellow is?

### Response:
The product is "Adult Ballet Tutu Yellow".

### Instruction:
Provide a list of all the items in the cart

### Input:
What items are in the cart?

### Response:
The items in the cart are "Adult Ballet Tutu Yellow", "Adult Ballet Tutu Black", "Adult Ballet Tutu Black", "Adult Ballet Tutu Black", "Adult Ballet Tutu Black", "Adult Ballet Tutu Black", "Adult Ballet Tutu Black", "Adult Ballet Tutu Black", "Adult Ballet Tutu Black", "Adult Ballet Tutu Black", "Adult Ballet Tutu Black", "Adult Ballet Tutu Black", "


### 3.3. Carregamento e Pré-processamento do Dataset

O dataset "The AmazonTitles-1.3MM" foi carregado e pré-processado para que os dados sejam adequados ao treinamento do modelo. As colunas **'input'** e **'output'** foram definidas como pontos de ajuste essenciais para o treinamento do modelo, representando, respectivamente, os dados de entrada e saída. Além disso, a coluna **'instruction'**, que contém a instrução padrão "Answer the user's question", foi adicionada para orientar o comportamento do modelo. A biblioteca *datasets* será usada para carregar os dados, com *pandas* auxiliando na manipulação adicional, caso necessário.
Para a limpeza do dataset, foi realizado a remoção de colunas irrelevantes, duplicatas, valores nulos e registros cujo conteúdo esteja vazio (apesar de conter demais informações), além disso as funções de limpeza eliminam entidades HTML e caracteres especiais.

In [ ]:
DATASET_PATH = '/content/drive/MyDrive/FIAP/fase-3/datasets/trn.json'

df = pd.read_json(dataset_path, lines=True)
df = df.drop(['uid', 'target_ind', 'target_rel'], axis=1)
df = df.rename(columns={'title': 'input', 'content': 'output'})
df['instruction'] = "Answer the user's question"
print(df.shape)
# Remover duplicatas
df = df.drop_duplicates(subset=['input', 'output'], keep='first')
print(df.shape)
# Remover registros com valores nulos
df = df.dropna(subset=['input', 'output'])
print(df.shape)
# Remover registros com conteúdo vazio
df = df[df['output'] != '']
print(df.shape)

# Função para remover entidades HTML e caracteres desnecessários
def limpar_html_entities(texto):
    # Substituir entidades HTML por seus equivalentes
    texto = re.sub(r'&#x201C;', '"', texto)  # Aspas de abertura
    texto = re.sub(r'&#x201D;', '"', texto)  # Aspas de fechamento
    texto = re.sub(r'&#x2019;', "'", texto)  # Apóstrofo
    texto = re.sub(r'&#x2026;', '...', texto)  # Reticências
    texto = re.sub(r'&#x2014;', '—', texto)  # Hífen longo (em dash)
    texto = re.sub(r'&#x2013;', '-', texto)  # Hífen curto (en dash)
    texto = re.sub(r'&quot;', '"', texto)  # Aspas duplas
    texto = re.sub(r'&apos;', "'", texto)  # Aspas simples
    texto = re.sub(r'&amp;', '&', texto)  # E comercial (&)
    texto = re.sub(r'&lt;', '<', texto)  # Menor que (<)
    texto = re.sub(r'&gt;', '>', texto)  # Maior que (>)
    texto = re.sub(r'&nbsp;', ' ', texto)  # Espaço não separável
    texto = re.sub(r'&copy;', '©', texto)  # Símbolo de copyright
    texto = re.sub(r'&reg;', '®', texto)  # Símbolo de marca registrada
    texto = re.sub(r'&euro;', '€', texto)  # Símbolo de euro
    texto = re.sub(r'&pound;', '£', texto)  # Símbolo de libra esterlina
    texto = re.sub(r'&yen;', '¥', texto)  # Símbolo de iene
    texto = re.sub(r'&cent;', '¢', texto)  # Símbolo de centavo
    texto = re.sub(r'&deg;', '°', texto)  # Símbolo de grau
    texto = re.sub(r'&hellip;', '...', texto)  # Reticências
    texto = re.sub(r'&ndash;', '-', texto)  # Hífen curto (en dash)
    texto = re.sub(r'&mdash;', '—', texto)  # Hífen longo (em dash)

    # Remover qualquer outra entidade HTML que não tenha sido substituída
    texto = re.sub(r'&[a-zA-Z0-9#]+;', '', texto)

    return texto


def limpar_texto(texto):
    # Remover caracteres especiais, exceto pontuações e números
    texto = re.sub(r'[^a-zA-Z0-9.,!?\'"()\s]', '', texto)
    # Remover espaços extras
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

# Aplicar a função de limpeza nos títulos e descrições
df['output'] = df['output'].apply(limpar_html_entities).apply(limpar_texto)

**OBSERVAÇÃO**: Devido a problemas de memória com as instancias do Collab, optamos por seguir com o tratamento dos dados na máquina local (com o script acima), e utilizar o dataset tratado salvo no drive.

 Nesta etapa, realizamos a formatação dos prompts para o fine-tuning do modelo, usando as colunas 'instruction', 'input' e 'output' como pontos de ajuste para o treinamento. A função `formatting_prompts_func` formata esses campos em uma string de prompt, utilizando o modelo alpaca_prompt, e adiciona o token de fim de sequência (EOS_TOKEN) ao final de cada texto. A biblioteca datasets carrega o dataset a partir do arquivo JSON, e a função de formatação é aplicada em lote ao dataset para preparar os dados adequadamente para o treinamento do modelo.

In [ ]:
DATASET_PATH = '/content/drive/MyDrive/FIAP/fase-3/datasets/trn_formated.json'
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass
dataset = load_dataset("json", data_files=DATASET_PATH, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1467983 [00:00<?, ? examples/s]

### 3.4 Configuração e Treinamento


Para configurarmos o modelo para o fine-tuning utilizando a abordagem de *Parameter Efficient Fine-Tuning (PEFT)*. A função `FastLanguageModel.get_peft_model` aplica ajustes de hiperparâmetros essenciais para otimizar o treinamento.

- **`r = 16`**: Este parâmetro define a quantidade de camadas de baixa-rank (LoRA) a serem utilizadas. O valor de 16 foi escolhido, pois oferece um bom equilíbrio entre desempenho e eficiência computacional, permitindo que o modelo aprenda representações úteis sem sobrecarregar os recursos.

- **`lora_alpha = 16`**: O valor de alpha controla a influência do ajuste de LoRA em relação ao modelo base. O valor de 16 é sugerido, pois fornece uma modulação adequada para ajustar os parâmetros sem distorcer as representações aprendidas anteriormente pelo modelo.

- **`lora_dropout = 0`**: Embora a técnica de *dropout* possa ser utilizada para prevenir overfitting, dado o cenário, identificamos que o valor de 0 é o mais otimizado.

Essas escolhas de hiperparâmetros são fundamentadas na busca por um equilíbrio entre eficiência computacional e a capacidade de aprendizado do modelo, visando obter um desempenho ideal no fine-tuning, principalmente se considerarmos as limitacoes de processamento de instancias com GPU do Google Collab.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Configuramos também o adapter de treino usando a classe `SFTTrainer` para o fine-tuning do modelo.

- **`max_seq_length`**: Definido para evitar sobrecarga de memória, garantindo que as sequências de entrada caibam na GPU. O modelo será capaz de processar até 128 tokens em uma única entrada.

- **`learning_rate = 1e-5`**: Selecionado para proporcionar estabilidade no treinamento, evitando oscilações excessivas. Um valor inadequado pode levar a overfitting ou underfitting.

- **`max_steps = 120`**: Limitado para se adequar ao tempo de execução do Colab, permitindo um ajuste eficaz do modelo sem overfitting.

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 120,
        learning_rate = 1e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()

Map (num_proc=2):   0%|          | 0/1467983 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,467,983 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.224700
2,3.153600
3,3.129800
4,3.068500
5,2.710700
6,2.656800
7,2.389900
8,2.355100
9,1.992100
10,1.894700


In [ ]:
def generate_response(question):
    alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    {}

    ### Input:
    {}

    ### Response:
    {}"""

    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Answer the user's question", # instruction
            question, # input
            "", # output
        )
    ], return_tensors = "pt").to("cuda")

    text_streamer = TextStreamer(tokenizer)
    _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

In [ ]:
generate_response("What product Adult Ballet Tutu Yellow is?")

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Answer the user's question

    ### Input:
    What product Adult Ballet Tutu Yellow is?

    ### Response:
    	Our adult ballet tutu is made of a soft, stretchy, nylon material and is 100% machine washable. The ballet tutu is made in 3 sizes. Size 1 is for children 2-4, Size 2 is for children 5-7, and Size 3 is for children 8-12. The tutu is available in 4 colors: pink, yellow, white, and black.<|end_of_text|>


In [ ]:
generate_response("What product 'My Life with the Wave' is?")

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Answer the user's question

    ### Input:
    What product 'My Life with the Wave' is?

    ### Response:
     My Life with the Wave is a new book by the author of the bestselling memoir, My Life with Bob. In this book, Wendy Wasserstein tells the story of her life with the wave--the wave that broke upon the shore of her consciousness in 1989, the wave that changed her life forever. In her usual witty, humorous, and honest way, Wasserstein explores the impact of her illness on her life, her career, and her family, and she also explores the impact of her illness on the people around her.<|end_of_text|>


In [ ]:
generate_response("What product The Biggest Bed in the World is?")

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Answer the user's question

    ### Input:
    What product The Biggest Bed in the World is?

    ### Response:
     The Biggest Bed in the World  is a hilarious book about the world's biggest bed.<|end_of_text|>


### 3.5. Salvar o Modelo Treinado
Por fim, podemos salvar o modelo para que seja utilizado futuramento sem a necessidade de um novo processo de treinamento.

In [ ]:
trained_model_path = '/content/drive/MyDrive/FIAP/fase-3/models/v4/lora_model'

model.save_pretrained(trained_model_path) # Local saving
tokenizer.save_pretrained(trained_model_path)

('/content/drive/MyDrive/FIAP/fase-3/models/v2/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/FIAP/fase-3/models/v2/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/FIAP/fase-3/models/v2/lora_model/tokenizer.json')

## 4. Conclusão dos Resultados

Os resultados obtidos foram satisfatórios, considerando as limitações de hardware do Google Colab. O modelo demonstrou uma capacidade adequada de aprendizado e generalização, validando a escolha dos parâmetros. A abordagem adotada permitiu um fine-tuning eficaz, equilibrando desempenho e eficiência dentro do ambiente restrito do Colab. Esses resultados indicam que, mesmo com restrições, é possível alcançar um desempenho competitivo no treinamento de modelos, principalmente se comparados as respostas antes e depois do fine tuning.